# 01. Activity Driven Networks

In this notebook, we will create a simple python code that generates a temporal network considering the activity driven framework.

In [ ]:
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import networkx as nx

### 1. Sample Activities

As a first exercise, we define a function that samples the activity from a power law distribution:


$$F(a) \sim a^{-\alpha}$$

In [ ]:
def sample_activity(eps, alpha):
    """
    This function samples the activity from a distribution F(a) ~ a^-alpha
    Parameters
    ----------
        @param eps: the minimum activity value
        @param alpha: the exponent of the power law distribution
    Returns
    -------
        @return: the sampled activity
    """
    y = random.random()
    a = (y + (1 - y) * (eps ** (1 - alpha))) ** (1 / (1 - alpha))
    return a

We test the function sampling $10^6$ activities considering a distribution with $\alpha=2.1$ and the minimum activity value $\epsilon=10^{-3}$ (to avoid divergence near the origin):

In [ ]:
# sample 10**6 activities and plot their distribution


### 2. Simulate Network (Random Partner Selection)

As a second exercise, we define a functions that generates the temporal network using the activity driven framework with random partner selection.


In [ ]:
def sample_neighbours(index, m, N):
    """
    This function samples the neighbours
    Parameters
    ----------
        @param index: source node index
        @param m: number of neighbours to sample
        @param N: total number of nodes
    Returns
    -------
        @return: the sampled neighbours
    """
    return 0


def get_adn_network(T, N, eps, alpha, m):
    """
    This function generates a temporal network using the activity
    driven framework
    Parameters
    ----------
        @param T: number of timesteps
        @param N: total number of nodes
        @param eps: the minimum activity value
        @param alpha: the exponent of the power law distribution
        @param m: number of neighbours to sample
    Returns
    -------
        @return: the total aggragated network, the timestep networks, 
                 and the time aggregated networks at different timesteps
    """
    # initialize activities
    
    # simulate 

    return 0 


def plot_network(G, k, ax=None):
    """
    This function plots a given network
    Parameters
    ----------
        @param G: the network to plot (nx.Graph object)
        @param k: plotting parameter
        @param ax: axes object, defaults to None
    """
    if ax == None:
        fig, ax = plt.subplots(dpi=300)

    d = dict(G.degree)
    nx.draw(G, 
            nodelist=d.keys(), 
            pos=nx.spring_layout(G, k=k),
            width=0.1, 
            node_size=[v for v in d.values()], 
            node_color=[v for v in d.values()], 
            cmap=sns.color_palette("crest", as_cmap=True), 
            ax=ax)

We test the functions generating a network with $5,000$ nodes over 20 timesteps and plotting the time-varying network at different timesteps:

In [ ]:
#... = get_adn_network(T=20, N=5000, eps=10**-3, alpha=2.1, m=10)
fig, axes = plt.subplots(dpi=300, ncols=3, figsize=(10,3))

Additionally, we also plot the aggregated network at different time steps:

In [ ]:
fig, axes = plt.subplots(dpi=300, ncols=3, figsize=(10,3))

### 3. Simulate Network (Communities)

As a third exercise, we define a functions that generates the temporal network using the activity driven framework with communities.

For simplicity, we will assume equal communities with fixed size.

In [ ]:
def sample_neighbours_community(index, m, nodes_list):
    """
    This function samples the neighbours in the case of communities
    Parameters
    ----------
        @param index: source node index
        @param m: number of neighbours to sample
        @param nodes_list: list of nodes in a community
    Returns
    -------
        @return: the sampled neighbours
    """
    return np.random.choice([idx for idx in nodes_list if idx != index], size=m)


def get_adn_network_community(T, N, eps, alpha, m, eta, community_size=10):
    """
    This function generates a temporal network using the activity
    driven framework with communities
    Parameters
    ----------
        @param T: number of timesteps
        @param N: total number of nodes
        @param eps: the minimum activity value
        @param alpha: the exponent of the power law distribution
        @param m: number of neighbours to sample
        @param eta: network modularity parameter
        @param community_size: size of each community
    Returns
    -------
        @return: the total aggragated network, the timestep networks, 
                 and the time aggregated networks at different timesteps
    """
    # initialize activities

    # initialize communities 
 
    # simulate 
    
    return 0

In [ ]:
# graph with three disconnected components
# ... = get_adn_network_community(T=20, N=6000, eps=10**-3, alpha=2.1, m=10, eta=1.0, community_size=2000)

# check its connected components